<div align='left' ><font size='70'>Setup</font></div>

In [ ]:
insert_learning_rate = 5e-5
insert_my_num_train_epochs = 20
my_train_batch_size = 64
my_weight_decay = 0.01
KLIST = [0, 1, 2, 3, 4]
list_model1and2 = [["roberta", "./drmodel6withmini5/best_model"], ]
pkl_file_list = ['sentence.pkl', 'big_label_list.pkl']
MultilLabel_MODE = True

from baseCode import jie, perfermance, send2me, AListinB  # type: ignore
from simpletransformers.classification import ClassificationModel  # type: ignore
import pandas as pd
import glob
import numpy as np
from sklearn import preprocessing
import pickle, itertools, copy
import seaborn as sns  # type: ignore
from numpy.random import randn
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib
import torch  # type: ignore
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, plot_confusion_matrix
from simpletransformers.classification import ClassificationModel, ClassificationArgs,
    MultiLabelClassificationModel  # type: ignore
from simpletransformers.model import TransformerModel  # type: ignore
from collections import Counter
import smtplib
from email.mime.text import MIMEText
from email.header import Header
import os

os.chdir(r"/home/anaconda3/data/V10")

pkl_file = open(pkl_file_list[0], 'rb')
train_test_5fold_list = pickle.load(pkl_file)
pkl_file.close()
pkl_file = open(pkl_file_list[1], 'rb')
big_label_list = pickle.load(pkl_file)
pkl_file.close()
print(big_label_list)

<div align='left' ><font size='70'>Model Training</font></div>

In [ ]:
for single_model1and2 in list_model1and2:
    model1=single_model1and2[0]
    model2=single_model1and2[1]
    print('🟥model',model1,'♦️topic',model2)
    for selected_k in KLIST:
        index_of_kfold=selected_k
        print("🎉kfold working on:",index_of_kfold)
        train_df=train_test_5fold_list[index_of_kfold][0]
        eval_df=train_test_5fold_list[index_of_kfold][1]
        i_X_train_epochs=1
        My_num_labels= int(len(big_label_list))
        print('label number:',My_num_labels)
        my_best_model_dir='clfmodel/'+str(selected_k)+'/best_model' 
        myargs={'fp16': False,\
                'best_model_dir':my_best_model_dir,\
                'learning_rate': insert_learning_rate,\
                'max_seq_length': 300,\
                'num_train_epochs':insert_my_num_train_epochs, \
                'train_batch_size': my_train_batch_size,\
                'reprocess_input_data': True,\
                'save_eval_checkpoints': False,\
                'save_model_every_epoch': False,\
                'save_optimizer_and_scheduler': True,\
                'overwrite_output_dir': True,\
                'no_cache': True,\
                'no_save': False,\
                'silent': False,\
                'evaluate_during_training':True,\
                'evaluate_during_training_steps':100,\
                'evaluate_during_training_verbose':True,\
                "evaluate_during_training_silent":True,\
                'n_gpu':2,\
                "wandb_project": "simpletransformers",\
                "wandb_kwargs": {"name": "paper best hyperparameter"},\
                "weight_decay":my_weight_decay,\
                "warmup_steps":78,\
                "scheduler": "linear_schedule_with_warmup",
                }
        if MultilLabel_MODE==True:
            model=MultiLabelClassificationModel(model1, model2,num_labels=int(My_num_labels),args=myargs,)
        #if MultilLabel_MODE==False:
            #model=ClassificationModel(model1, model2, num_labels=int(My_num_labels+1),#args=myargs)
        model.train_model(train_df,eval_df=eval_df)

<div align='left' ><font size='70'>Evaluation</font></div>

In [ ]:
KLIST=[0,1,2,3,4]
'''
pkl_file_list=['merged_train_test_5fold_list.pkl','merged_big_label_list.pkl']
pkl_file = open(pkl_file_list[0], 'rb')
train_test_5fold_list = pickle.load(pkl_file)
pkl_file.close()
'''
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix
y_true_list_full=[]
y_pred_list_full=[]
for selected_k in KLIST:
    eval_df=train_test_5fold_list[selected_k][1]
    my_best_model_dir='clfmodel/'+str(selected_k)+'/best_model'
    n_model_args = {
    "threshold": 0.5
    }
    if MultilLabel_MODE==True:
        model=MultiLabelClassificationModel("roberta", 
        my_best_model_dir,args=n_model_args,)
    if MultilLabel_MODE==False:
        model=ClassificationModel("roberta", 
        my_best_model_dir,args=n_model_args,)
        result, model_outputs, wrong_predictions = model.eval_model(eval_df)
        print('：',result)
    predictions, raw_outputs = model.predict(eval_df['text'].values.tolist())
    y_true_list=eval_df['labels'].values.tolist()
    y_pred_list=predictions
    y_true_list_full.extend(y_true_list)
    y_pred_list_full.extend(y_pred_list)
    print(classification_report(np.array(y_true_list), np.array(y_pred_list),target_names= big_label_list))
y_true = np.array(y_true_list_full)
y_pred = np.array(y_pred_list_full)
text_full=[]
for selected_k in KLIST:
    eval_df=train_test_5fold_list[selected_k][1]
    text_full.extend(eval_df['text'].values.tolist())
g=0
t=0
for i in range(len(y_true_list_full)):   
    if y_pred_list_full[i]==y_true_list_full[i]:
        #print(y_pred_list[i],y_true_list[i])
        g=g+1
print('acc',g/len(y_pred_list_full))
print(classification_report(y_true, y_pred,target_names= big_label_list))